In [4]:
import pandas as pd
import numpy as np
import json

import functions

In [5]:
# load and unpack

fp = 'data/news/raw/netflix_stock_news.json'

with open(fp) as json_file:
    data = json.load(json_file)
    news_df = pd.read_json(fp)
articles = functions.newsapiArticleUnpack(news_df)
articles.head()

,author,content,description,publishedAt,source,title,url,urlToImage
0,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...
1,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...
2,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...
3,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...
4,newsfeedback@fool.com (Anders Bylund),Media-streaming veteran Netflix(NFLX -1.87%) i...,The digital video maven will report earnings n...,2022-07-12T14:05:22Z,"{'id': None, 'name': 'Motley Fool'}",Should You Buy Netflix Stock Before Next Week'...,https://www.fool.com/investing/2022/07/12/shou...,https://g.foolcdn.com/editorial/images/689628/...


In [6]:
# let's just grab the relevant text and date
articles_text = articles[['title', 'description', 'content']]
articles_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        126 non-null    object
 1   description  126 non-null    object
 2   content      126 non-null    object
dtypes: object(3)
memory usage: 3.9+ KB


In [7]:
# process text and merge it
processed_text = functions.text_preprocessing(articles_text)
articles_processed = pd.concat([articles, processed_text], axis=1)

In [8]:
# # export as txt file for finBERT sentiment classification
# # writing from pandas is not working great, let's try from list to file
# out_fp = '../data/news/to_classify/'

# for column in articles_processed[['title_processed','description_processed', 'content_processed']]:
#     articles_processed[column].to_csv(out_fp+str(column)+'.txt', index=False, encoding='utf-8-sig')

In [9]:
# # V2 file writing
# out_fp = '../data/news/to_classify/'
# # find each set of sentences and convert to list
# for column in articles_processed[['title_processed','description_processed', 'content_processed']]:
#     temp_list = list(articles_processed[column])
#     # start writing to file
#     with open(out_fp+str(column)+'.txt', 'w') as fp:
#         for item in temp_list:
#             # write each item on a new line
#             fp.write("%s\n\n" % item)
#         print('Done')

In [22]:
articles_text

,title,description,content
0,Why Is Everyone Talking About Netflix Stock?,"""Stranger Things"" has captured over 1 billion ...",Netflix(NFLX -1.21%) said that the latest seas...
1,Why Netflix Stock Fell 4% Today,Investors are bracing for more bad news from t...,What happened\r\nNetflix(NFLX -4.09%) sharehol...
2,Why Is Everyone Talking About Netflix Stock?,"""Stranger Things"" has captured over 1 billion ...",Netflix(NFLX -1.21%) said that the latest seas...
3,Why Netflix Stock Fell 4% Today,Investors are bracing for more bad news from t...,What happened\r\nNetflix(NFLX -4.09%) sharehol...
4,Should You Buy Netflix Stock Before Next Week'...,The digital video maven will report earnings n...,Media-streaming veteran Netflix(NFLX -1.87%) i...
...,...,...,...
121,Can Netflix Stock Recover After Subscriber Los...,Netflix is a tech company because the key to i...,"cyano66\r\nAs I write this, Netflix (NASDAQ:NF..."
122,"Netflix Shares Rose 28.6% Last Month, But the ...",Here's why Netflix gained nearly 30% last mont...,What happened\r\nShares of Netflix(NFLX -2.12%...
123,This Streaming Stock Is Poised for Monster Gro...,Peacock's moment might have arrived.,Streaming services saw a bump in sign-ups thro...
124,Thinking about trading options or stock in NVI...,"NEW YORK, Aug. 4, 2022 /PRNewswire/ -- Investo...","NEW YORK, Aug. 4, 2022 /PRNewswire/ -- Investo..."


In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
import os

model_fp = 'models/classifier_model/finbert-sentiment'
text_fp = 'data/news/to_classify/content_processed.txt'
output_fp = '/Users/niclapenna/Documents/GitHub/lhl_capstone/data/news/classified'

with open(text_fp,'r') as f:
    text = f.read()

finbert = AutoModelForSequenceClassification.from_pretrained(model_fp,num_labels=3,cache_dir=None)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

output = "predictions.csv"
# predict(text,model,write_to_csv=True,path=os.path.join(output_fp,output))

In [41]:
# Use pytorch transformers instead
text_dict = {}

for column in articles_text[['title','description', 'content']]:
    text_dict[column] = list(articles_text[column])

In [42]:
articles_text

,title,description,content
0,Why Is Everyone Talking About Netflix Stock?,"""Stranger Things"" has captured over 1 billion ...",Netflix(NFLX -1.21%) said that the latest seas...
1,Why Netflix Stock Fell 4% Today,Investors are bracing for more bad news from t...,What happened\r\nNetflix(NFLX -4.09%) sharehol...
2,Why Is Everyone Talking About Netflix Stock?,"""Stranger Things"" has captured over 1 billion ...",Netflix(NFLX -1.21%) said that the latest seas...
3,Why Netflix Stock Fell 4% Today,Investors are bracing for more bad news from t...,What happened\r\nNetflix(NFLX -4.09%) sharehol...
4,Should You Buy Netflix Stock Before Next Week'...,The digital video maven will report earnings n...,Media-streaming veteran Netflix(NFLX -1.87%) i...
...,...,...,...
121,Can Netflix Stock Recover After Subscriber Los...,Netflix is a tech company because the key to i...,"cyano66\r\nAs I write this, Netflix (NASDAQ:NF..."
122,"Netflix Shares Rose 28.6% Last Month, But the ...",Here's why Netflix gained nearly 30% last mont...,What happened\r\nShares of Netflix(NFLX -2.12%...
123,This Streaming Stock Is Poised for Monster Gro...,Peacock's moment might have arrived.,Streaming services saw a bump in sign-ups thro...
124,Thinking about trading options or stock in NVI...,"NEW YORK, Aug. 4, 2022 /PRNewswire/ -- Investo...","NEW YORK, Aug. 4, 2022 /PRNewswire/ -- Investo..."


In [43]:
sent_val = []

labels = {0:'neutral', 1:'positive',2:'negative'}

for column in text_dict.keys():
    for sentence in text_dict[column]:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True)
        outputs = finbert(**inputs)[0]
        val = labels[np.argmax(outputs.detach().numpy())]
        print(sentence, '#####', val)
        sent_val.append(val)

Why Is Everyone Talking About Netflix Stock? ##### negative
Why Netflix Stock Fell 4% Today ##### negative
Why Is Everyone Talking About Netflix Stock? ##### negative
Why Netflix Stock Fell 4% Today ##### negative
Should You Buy Netflix Stock Before Next Week's Earnings Report? ##### negative
Is Netflix Stock A Sell Before Upcoming Earnings? Consider Short-Term Vs. Long-Term Outlook ##### negative
Should You Buy Netflix Stock Before Next Week's Earnings Report? ##### negative
Is Netflix Stock A Sell Before Upcoming Earnings? Consider Short-Term Vs. Long-Term Outlook ##### negative
Thinking about trading options or stock in Advanced Micro Devices, Exxon Mobil, General Mills, Amazon, or Netflix? ##### positive
Is Netflix Stock Undervalued? ##### negative
Is Netflix Stock a Buy Before Earnings? ##### negative
Here's Why Netflix Stock Could Soar Next Week ##### negative
Thinking about trading options or stock in Advanced Micro Devices, Exxon Mobil, General Mills, Amazon, or Netflix? ##### 

IndexError: index out of range in self